In [ ]:
import numpy as np
import networkx as nx
import random
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import itertools
import pickle
import operator
import copy
import os
%matplotlib notebook

### Identify vulnerable individuals
We define vulnerability using two metrics, (1) fraction of times a cascade reached nodes, (2) time (steps) for a cascade to reach a node.

__Procedure__
1. initialize random network with N nodes
2. select n seeds at random
3. run spreading process, truncate spread after $t$ steps or after process has run its course (if $p_{\text{infection}} < 1$)
4. identify nodes which arent reached
5. repeat process m times
6. construct distribution of vulnerability (fraction of time node was not reached)

__Spreading dynamics__ - Independent Cascade Model
1. At $t=0$ all nodes are inactive, except seed nodes
2. Each activated node i contacts all its neighbors j and, with an independent probability p, tries to activate each of those nodes that have been never activated during previous stages of the dynamics
3. All nodes that tried to activate their neighbors at step (i) become inactive, and they cannot be activated again in subsequent stages of the dynamics. The process is iterated until no more active nodes are present

__How to select $p$?__
For a network the critical value $p_c$ separates the region of the phase-diagram where outbreaks are subextensive ($p < p_c$) from the supercritical phase ($p > p_c$) where outbreaks reach a finite fraction of the whole network.

The value of $p_c$ is determined as the position of the maximum of the susceptibility $⟨s^2⟩/⟨s⟩^2$ (where $⟨s^n⟩$ is the n-th moment of the outbreak size distribution computed for random initial single spreaders).

Read more in: _On the numerical study of percolation and epidemic critical
properties in networks_, Castellano and Pastor-Satorras (2016) [https://arxiv.org/pdf/1610.01375.pdf]

In [ ]:
def ICM_special(network,seeds,p_infection):
    '''
    Independent cascade model (this is a slightly adapted version of the cascade model, i.e. faster, than the one used in core_functions.py) 
    ------------------    
    Input:
    * network: directed opr undirected network
    * seeds: list of seed nodes or number of initial seeds to simulate the dynamics from
    * p: infection/spreading probability 
    ------------------
    Output: 
    * set of nodes which have been infected
    ------------------
    '''
    if type(seeds) != list:
        infected = set(random.sample(network.nodes(),seeds)) # infect seeds
    else:
        infected = set(seeds)

    activated = infected.copy()

    # run infection process
    while infected != set():
        new_infected = set()
        for node in infected:
            for neighbor in network.neighbors(node):
                if (random.random() < p_infection) and (neighbor not in activated):
                    new_infected.add(neighbor)
                    
                    # update sets
                    activated.add(neighbor)
            # update set
            activated.add(node)
        # set of newly infected nodes
        infected = new_infected.copy()
    
    # return list of infected/recovered nodes
    return activated

# Identify $p_c$ values
__Find a $p_c$ for each network__

In [ ]:
# parameters 
gamma = 2.5 # to use when loading SF networks
load_path = 'some_path_somewhere'
save_path = 'other_path_somewhere'

# iterate over the 100 realization of synthetic networks
for id_ in range(100): 
    Sp = []
    print id_
    # load synthetic network
    G = nx.Graph()
    #with open(load_path + '/ER%03d.csv' % id_) as f:
    with open(load_path + '/SF%03d_gamma%.1f.csv' % (id_,gamma)) as f:
        for line in f:
            i,j = map(int,line.strip().split(','))
            G.add_edge(i,j)
            
    Nn = len(G.nodes())
    
    for p in np.linspace(0.065,0.105,41): # SF
        print '\t %.3f' % p
        S = []
        for i in range(10*Nn):
            # spreading dynamics
            recovered = ICM(G,1,p)
            S.append(len(recovered)/float(Nn))
        Sp.append((p,np.mean(np.array(S)**2)/np.mean(S)**2))
    
    #pkl_file = open(save_path + '/ER_Pc_%03d.pkl' % id_,'w')
    pkl_file = open(save_path+ '/SF_Pc_%03d_gamma2.5.pkl' % id_,'w')
    pickle.dump(Sp,pkl_file)
    pkl_file.close()

# plot $p_c$ curves

In [ ]:
import cPickle as pickle
import os

# locate files
Pc = dict()
path = '/Users/vsekara/data/last_mile/p_c/'
for type_ in ['ER','gamma2.5']:
    files = [f for f in os.listdir(path) if type_ in f]

    # open data
    Pc[type_] = []
    for ff in files:
        pkl_file = open(path + ff,'r')
        dat = pickle.load(pkl_file)
        pkl_file.close()

        Pc[type_].append(round(max(dat,key=operator.itemgetter(1))[0],4))

__New version of the plot__

In [ ]:
plt.figure(figsize=(8,3))
plt.subplot(121)
plt.hist(Pc['ER'],bins=np.arange(0.26,0.301,0.002),color='black',rwidth=0.9)
plt.xlabel('$p_c$')
plt.ylabel('ER Networks')

plt.subplot(122)
plt.hist(Pc['gamma2.5'],bins=np.arange(0.05,0.15,0.005),color='black',rwidth=0.9)
plt.xlabel('$p_c$')
plt.ylabel('SF Networks')

plt.tight_layout()
plt.show()